Para baseline:
1) Criar o dataset sintético
2) Injeta missing sob algum mecanismo
3) Imputação com os algoritmos e avaliação dos resultados

Para attribute e label noise (primeiro separadamente):
1) Dado o mesmo dataset sintético do baseline
2) Injeta Attribute/Label noise com alguma porcentagem (por exemplo, 10%)
3) Gera os missing com mesmo mecanismo
4) Imputação com os algoritmos e avaliação dos resultados

Por fim, 
1) Dado o mesmo dataset
2) Injeta Attribute/Label noise com alguma porcentagem (por exemplo, 10%)
3) Gera os missing com mesmo mecanismo
4) Realiza um pré-processamento do noise
5) Imputação com os algoritmos e avaliação dos resultados
Hipótese: espera-se que o ruído degrade a performance  na imputação/classificação

### Funções

In [ ]:
from sklearn.datasets import make_classification, make_blobs
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from collections import Counter
from imblearn.under_sampling import EditedNearestNeighbours

# Gaussian Noise
def generate_gaussian_noise(dataset:pd.DataFrame, column:str)-> float:
    """Função para gerar um ruído Gaussiano respeitando o máximo e mínimo 
    da coluna do dataset
    
    Args:
        dataset (pd.DataFrame): pandas Dataframe
        column (str): a coluna do dataset para gerar o noise
    
    Returns:
        noise (float): ruído gaussiano para a coluna do dataset
    """

    col_mean = dataset[column].mean()
    col_std = dataset[column].std()
    max_val = dataset[column].max()
    min_val = dataset[column].min()
    noise = np.random.normal(col_mean, col_std)

    if noise > max_val:
        noise = max_val
    elif noise < min_val:
        noise = min_val

    return noise

def cria_dataset_sintético()->pd.DataFrame:
    """Função para criar um dataset sintético com 500 observações em 5 atributos.
    O dataset sintético é balanceado, com 3 features informativas e 1 cluster por classe;
    Sem nenhuma troca de classe (flip_y = 0.0);
    e uma distância de classe 3.0.
    """
    
    X_synthetic, y_synthetic = make_classification(n_samples=500, 
                                                    n_features=5, 
                                                    n_informative=5, 
                                                    n_redundant=0, 
                                                    n_classes=2, 
                                                    random_state=42, 
                                                    n_clusters_per_class=1, 
                                                    flip_y=0.0,
                                                    class_sep=0.2, 
                                                    )
    
    df = pd.DataFrame(X_synthetic, columns=["Att1", "Att2", "Att3", "Att4", "Att5"])
    df["target"] = y_synthetic

    return df

def generate_random_noise(data, length):
    
    random_numbers = np.random.random(length)

    max_val = max(data.max())
    min_val = min(data.min())
 
    for cont in range(len(random_numbers)):
        if random_numbers[cont] > max_val:
            random_numbers[cont] = max_val       
        elif random_numbers[cont] < min_val:
            random_numbers[cont] = min_val

    return random_numbers
    

def injeta_ruido_dataset(data:pd.DataFrame, noise_rate:float, target:bool = True):
    """Função para gerar ruído Gaussiano em pandas Dataframe.

    Args:
        dados_sinteticos (pd.DataFrame): os dados que receberão o ruído
        noise_rate (float): a porcentagem de geração do ruído
        target (bool): Uma flag para indicar o drop da coluna target

    Returns:

    """
    y = data["target"].copy()
    
    if target:
        data = data.drop(columns="target")

    original_shape = data.shape
    n = data.shape[0]
    p = data.shape[1]
    N = round(n * p * noise_rate)

    array_values = data.values
    pos_noise = np.random.choice(n*p, N, replace=False)
    array_values = array_values.flatten()
    array_values[pos_noise] = generate_random_noise(data, N)
    array_values = array_values.reshape(original_shape)
    
    data_noise = pd.DataFrame(array_values, columns=data.columns)
    data_noise['target'] = y
    return data_noise

def flip_label(dados:pd.DataFrame, noise_level:float)->pd.DataFrame:
    """
    Função para inverter a label baseada em uma porcentagem.
    """

    dataset = dados.copy()
    y = dataset["target"]
    invert_label = {1:0,
                    0:1}

    N = round(noise_level*len(y))

    pos = np.random.choice(range(len(y)), N, replace=False)
    dataset.loc[pos, "target"] = dataset.loc[pos, "target"].map(invert_label)

    return dataset

### Experimentos

In [ ]:
path = "C:\\Users\\Mult-e\\Desktop\\@Codigos\\Datasets sintéticos\\"
file = "dados_sinteticos.csv"

# dados_sinteticos = cria_dataset_sintético()
# dados_sinteticos.to_csv(path+file, index=False)

In [ ]:
# Attribute Noise com ruído Gaussiano

dados_sinteticos = pd.read_csv(path+file)

ruido = injeta_ruido_dataset(dados_sinteticos, 0.05)
nome = "dadosAtt05"


for coluna in ruido:
    plt.plot(ruido[coluna], label=f'Depois - {coluna}', linestyle = "--", color = "blue")

for coluna in dados_sinteticos:
    plt.plot(dados_sinteticos[coluna], label=f'Antes - {coluna}', color = "black")


ruido.to_csv(f"C:\\Users\\Mult-e\\Desktop\\@Codigos\\Datasets sintéticos\\{nome}.csv", index=False)

In [ ]:
# Label Noise
noise_rate = 0.20

label_noise_df = flip_label(dados_sinteticos, noise_rate)
label_noise_df.to_csv(path+"dadosLabel20.csv")